In [ ]:
!pip install opencv-contrib-python==4.4.0.40 --force-reinstall

     |████████████████████████████████| 55.6 MB 153 kB/s 
     |████████████████████████████████| 15.7 MB 29.7 MB/s 
Reason for being yanked: Release deprecated
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import cv2
cv2.__version__

'4.4.0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Baixa o repositório da Darknet:
%cd "content/"

!git clone https://github.com/AlexeyAB/darknet

# Configura o MAKEFILE para o uso do CUDA (DEPENDE DE GPGPU):
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

# Verifica o CUDA:
!/usr/local/cuda/bin/nvcc --version

# Instala a Darknet:
!make

[Errno 2] No such file or directory: 'content/'
/content
Cloning into 'darknet'...
remote: Enumerating objects: 15412, done.
remote: Total 15412 (delta 0), reused 0 (delta 0), pack-reused 15412
Receiving objects: 100% (15412/15412), 14.04 MiB | 19.12 MiB/s, done.
Resolving deltas: 100% (10356/10356), done.
/content/darknet
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void dra

In [ ]:
%cd "/content/drive/MyDrive/IC_YOLO_MESCLADO/treinamento"

! /content/darknet/darknet detector train data/directives.data model.cfg backup/model_last.weights -dont_show -map

A saída de streaming foi truncada nas últimas 5000 linhas.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 341693, rewritten_bbox = 0.035119 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.848749), count: 43, class_loss = 0.729064, iou_loss = 169.223053, total_loss = 169.952118 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.765313), count: 9, class_loss = 0.375535, iou_loss = 11.622462, total_loss = 11.997996 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.000000), count: 1, class_loss = 0.000004, iou_loss = 0.000000, total_loss = 0.000004 
 total_bbox = 341745, rewritten_bbox = 0.035114 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.795516), count: 26, class_loss = 1.749231, iou_loss = 100.665497, total_l

In [ ]:
from os import path, getcwd, chdir, listdir
import cv2
from numpy import argmax

%cd "/content/drive/MyDrive/IC_YOLO_MESCLADO/ativacao"

# Carrega a lista de imagens no path de ativação:
image_files = []
chdir(path.join("input"))
for filename in listdir(getcwd()):
    if filename.endswith(".jpg"):
        image_files.append(filename)
chdir("..")

# Carrega a rede neural convolucional e sua arquitetura:
net = cv2.dnn.readNet("networks/model.weights", "networks/model.cfg")

# Carrega as classes:
classes = []
with open("networks/model.names", 'r') as f:
    classes = f.read().splitlines()

# Executa a ativação em todo arquivo JPG dentro do path de ativação:
for image in image_files:
    # Carrega a imagem:
    img = cv2.imread("input/" + image)

    # Váriaveis e ponteiros de execução:
    height, width, _ = img.shape

    net.setInput(cv2.dnn.blobFromImage(img, (1 / 255), (415, 416), (0, 0, 0), swapRB=True, crop=False))
    boxes = []
    confidences = []
    class_ids = []

    # Detecção e armazenamento das instâncias dos objetos:
    for output in net.forward(net.getUnconnectedOutLayersNames()):
        for detection in output:
            scores = detection[5:]
            class_id = argmax(scores)
            confidence = scores[class_id]
            if (confidence > 0.5):
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - (w / 2))
                y = int(center_y - (h / 2))
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Desenha os retângulos, as classes e a taxa de confiança nos quadros:
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    if (len(indexes) > 0):
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i], 2))
            cv2.rectangle(img, (x, y), ((x + w), (y + h)), (0, 255, 0), 2)
            cv2.putText(img, (label + ' ' + confidence), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 255, 255), 1, cv2.LINE_4)

    # Salva a imagem de saída:
    cv2.imwrite(("output/" + image), img)

/content/drive/MyDrive/IC_YOLO_MESCLADO/ativacao
